# Lung Cancer Classification using Computerized Tomography
 Project developed by: **Eduardo Passos** [202205630](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202205630), **Pedro Fernandes** [202208347](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202208347) and **Rafael Pacheco** [202206258](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202206258)


### Index {#index} #############################################
1. [Project Introduction](#intro)
2. [Data Understanding](#understand)
3. [Feature Extraction](#ext)
4. [Feature Selection and Dimensionality Reduction](#reduct)
5. [Exploratory Data Analysis](#eda)
6. [Models](#models)
6.1. [Support Vector Machines](#svm)


## 1. Project Introduction and Motivation {#intro}
[Back to Index](#index)

This project focuses on classifying lung cancer using Computed Tomography (CT) scan data, using the [**LIDC-IDRI**](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=1966254) dataset. 

As referenced in the assignment, **lung cancer leads global cancer-related mortality rates**, as only 16% of cases are diagnosed as local-stage tumours. In these instances, patients have a five-year survival rate exceeding 50%. However, when diagnosed at an advanced stage, the five-year survival rate drops to just 5%. Therefore, **early diagnosis is crucial for improving survival outcomes**, and systems designed to assist with screening can be instrumental regarding this matter (statistics from [World Health Organization](https://www.iarc.who.int/featured-news/latest-global-cancer-data-cancer-burden-rises-to-18-1-million-new-cases-and-9-6-million-cancer-deaths-in-2018/) and [American Cancer Society](https://www.cancer.org/research/cancer-facts-statistics/all-cancer-facts-figures/cancer-facts-figures-2019.html)).

The goal is to **develop a machine learning model capable of accurately identifying lung cancer** (based on the provided medical imaging data) in early stages of the tumour, in order to prevent evolution of the disease. To support this, we have developed a program that **converts the dataset** (initially in DICOM format) into numpy array files (.npy), while extracting and storing key information in a CSV file.

---

## 2. Data Understanding {#understand}
[Back to Index](#index)

The [LIDC-IDRI](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=1966254) dataset **contains the imaging data of the lungs** from over 1000 patients. This dataset is part of a collection of datasets provided by [**The Cancer Imaging Archive**](https://www.cancerimagingarchive.net/browse-collections/) for public research. The medical imaging data is provided in [**DICOM**](https://en.wikipedia.org/wiki/DICOM) (.dcm), and contains both image data, and associated metadata. It also contains the annotations of the medical professionals regarding the subject. 

The nodule annotations in the dataset **reflect the radiologist's evaluation** and include key information such as nodule size and malignancy rating (on a scale from 1-5). However, the **malignancy classification may vary based on each radiologist's judgment** and reasoning. What this means is that it introduces a level of subjectivity, making the classification potentially ambiguous and not always straightforward.

The dataset contains noise, missing data, and file corruption. Some patients have incomplete annotations or no annotations at all, which complicates the training process since not all scans contain the necessary labels for nodule classification. 

In summary, not only many exceptions must be dealt with in the extratcion part, but also checked in the cleanup process.

---

## 3. Feature Extraction {#ext}
[Back to Index](#index)

This chapter explains the code behind the `feature_extraction_3D.py` script, designed to extract features from the dataset. This script uses a combination of libraries for medical image processing, radiomic feature extraction, and deep learning-based feature extraction using a [CNN (ResNet-18 model)](https://medium.com/analytics-vidhya/resnet-understand-and-implement-from-scratch-d0eb9725e0db). Below is an in-depth explanation of the "routines" that constitute the process.


### 1) Key Libraries

- **pylidc**: For the interaction with the LIDC-IDRI dataset.
- **SimpleITK**: For medical image processing tasks.
- **radiomics**: Provides various techniques to extract quantitative radiomic features from 3D medical imaging, like shape, texture, and intensity-based features.
- **torch and monai**: Used to train the neural network for feature extraction. Specifically, `monai.networks.nets.ResNet18`, which provides a pre-trained ResNet-18 CNN model designed for medical imaging tasks.

### 2) Command-Line Argument Parsing: 

The `parse_arguments()` function uses Python's `argparse` library to parse command-line arguments, allowing for flexible and customizable script behavior. The **arguments defined** are the following:

1. **`--output_dir`**: Specify the folder to save the extracted feature CSV files.
   
2. **`--log_file`**: Path to the file where the script will write all its logs and messages. 

3. **`--csv_file`** (Optional): Choose whether to save all features in a single CSV file or in 2, this is where you set the path. (individual CSVs are saved by default)

4. **`--scan_limit`**: Control how many scans the script processes. If 0 or a negative number, it will process all available scans.

5. **`--consensus_level`**: This sets the confidence level (between 0 and 1) for generating a consensus mask from multiple radiologist annotations.

6. **`--num_workers`**: Number of worker threads for parallel processing. 

7. **`--cnn_feature_dim`**: Defines the size of the CNN feature vector to extract, with a default set at 128 dimensions.

8. **`--model_path`**: Path to a pre-trained CNN model. If none is provided, the script will use a ResNet-18 model from the MONAI library as the fallback.

Parsed arguments can be easily accessed throughout the script.


### 3) Resampling and HU Normalization

To ensure reliability and standardization across scans, the 3D CT scans are resampled to a fixed voxel spacing using **trilinear interpolation**. This ensures that the scans from different machines or resolutions are brought to a consistent format.

Additionally, Hounsfield Units (HU), which represent the density values in CT images, are normalized. This involves two main functions:

- **`clamp_hounsfield_units()`**: Clamps HU values within a specific range (e.g., -1000 to 400) to focus on the regions of interest like lung tissue (excluding irrelevant areas like air or bones).
- **`normalize_intensity()`**: Further scales the intensity values for consistent input into the neural network model.


### 4) Bounding Box Calculation: 

After preprocessing, the `get_bounding_box()` determines the **bounding box** for each nodule in the scan:

- The **binary mask** representing the nodule is processed to find the smallest box that contains all `True` (or 1) values, which represent the nodule region.
- This is crucial for reducing the amount of data being processed, focusing only on the nodule area instead of the entire scan.


### 5) Nodule Extraction and Resizing

Once the bounding box is determined, the nodule is extracted from the original 3D scan. The extracted nodule is then resized to a consistent shape and size, making it ready for feature extraction. This is especially important for the CNN, which expects inputs of fixed dimensions.


### 6) Malignancy Evaluation

Each nodule in the LIDC-IDRI dataset is annotated by multiple radiologists, and each provides a score for malignancy (among other characteristics). The **median score** of malignancy is used as a target label for training the model. This helps in creating a more robust and reliable training dataset by minimizing bias from individual radiologists' evaluations.


### 7) CNN-Based Feature Extraction: ResNet-18

For deep learning-based feature extraction, a **ResNet-18 CNN model** is applied to each extracted nodule. ResNet-18 is a well-known architecture that introduces residual connections, enabling deeper networks to be trained efficiently.

- The script uses a **pre-trained ResNet-18 model** (either provided via `--model_path` or by default from MONAI).
- The model extracts **CNN features** of a predefined size (e.g., 128-dimensional), which are saved as part of the final feature set.


### 8) Radiomic Feature Extraction

In addition to CNN-based features, the script also uses the **radiomics library** to extract traditional radiomic features from the nodules. These features include:

- **Shape features**: Describing the geometric properties of the nodule (e.g., volume, elongation, flatness).
- **Texture features**: Quantifying the patterns and variations in pixel intensity within the nodule.
- **Intensity-based features**: Capturing the intensity distribution in the nodule region.


### 9) Feature Aggregation and Output

Once both CNN and radiomic features are extracted, they are **aggregated into a final CSV file**. Each row in the CSV file represents a nodule, meaning each nodule is evaluated independently, and helps on the case of a patient having different nodules.

This CSV file serves as the input for subsequent machine learning tasks.

The script is designed to save separate CSV files for each nodule by default. However, an option is provided to save a single CSV file containing all features via the `--csv_file` argument.

---

## 4. Feature Selection and Dimensionality Reduction {#reduct}
[Back to Index](#index)


## 5. Exploratory Data Analysis {#eda}
[Back to Index](#index)

